# Amazon Food Reviews - Classifier

In [6]:
# !pip install -U -q PyDrive
# from pydrive.auth import GoogleAuth
# from pydrive.drive import GoogleDrive
# from google.colab import auth
# from oauth2client.client import GoogleCredentials
# auth.authenticate_user()
# gauth = GoogleAuth()
# gauth.credentials = GoogleCredentials.get_application_default()
# drive = GoogleDrive(gauth)
# json_import = drive.CreateFile({'id':'1KpQKJnoX3XCpyW788srGxtO3nwZFJ4oE'})
# json_import.GetContentFile('database.sqlite')
# json_import = drive.CreateFile({'id':'1XRPgjWYDhIllbS4VWLGDIJ8-pcP7A_r4'})
# json_import.GetContentFile('final.sqlite')

In [7]:
%matplotlib inline

import sqlite3
import pandas as pd
import numpy as np
import nltk
nltk.download('stopwords')
import string
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
import re
from sklearn.utils import shuffle

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from nltk.stem.porter import PorterStemmer
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import FreqDist
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.callbacks import EarlyStopping

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Shubham\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
# using the SQLite Table to read data.
con = sqlite3.connect('./final.sqlite') 


#filtering only positive and negative reviews i.e. 
# not taking into consideration those reviews with Score=3
filtered_data = pd.read_sql_query("""
SELECT *
FROM Reviews
WHERE Score != 3
""", con) 


In [9]:
filtered_data.head()

,index,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,CleanedText
0,138706,150524,0006641040,ACITT7DI6IDDL,shari zychinski,0,0,positive,939340800,EVERY book is educational,this witty little book makes my son laugh at l...,b'witti littl book make son laugh loud recit c...
1,138688,150506,0006641040,A2IW4PEEKO2R0U,Tracy,1,1,positive,1194739200,"Love the book, miss the hard cover version","I grew up reading these Sendak books, and watc...",b'grew read sendak book watch realli rosi movi...
2,138689,150507,0006641040,A1S4A3IQ2MU7V4,"sally sue ""sally sue""",1,1,positive,1191456000,chicken soup with rice months,This is a fun way for children to learn their ...,b'fun way children learn month year learn poem...
3,138690,150508,0006641040,AZGXZ2UUK6X,"Catherine Hallberg ""(Kate)""",1,1,positive,1076025600,a good swingy rhythm for reading aloud,This is a great little book to read aloud- it ...,b'great littl book read nice rhythm well good ...
4,138691,150509,0006641040,A3CMRKGE0P909G,Teresa,3,4,positive,1018396800,A great way to learn the months,This is a book of poetry about the months of t...,b'book poetri month year goe month cute littl ...


In [10]:
# Give reviews with Score>3 a positive rating, and reviews with a score<3 a negative rating.
# def partition(x):
#     if x < 3:
#         return 'negative'
#     return 'positive'
def give_scores(score):
    score = np.asarray(score)
    score = np.where(score == 'positive', 1, 0)
    return score

#changing reviews with score less than 3 to be positive and vice-versa
actualScore = filtered_data['Score']
# positiveNegative = actualScore.map(partition) 
positiveNegative = give_scores(actualScore)
filtered_data['Score'] = positiveNegative

In [11]:
filtered_data['Score'].value_counts()

1    307061
0     57110
Name: Score, dtype: int64

In [12]:
#Sorting data according to ProductId in ascending order
sorted_data=filtered_data.sort_values('ProductId', axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')

#Deduplication of entries
final=sorted_data.drop_duplicates(subset={"UserId","ProfileName","Time","Text"}, keep='first', inplace=False)

final=final[final.HelpfulnessNumerator<=final.HelpfulnessDenominator]

## Text Preprocessing: Stemming, stop-word removal and Lemmatization.


In [13]:
# find sentences containing HTML tags
# i=0;
# for sent in final['Text'].values:
#     if (len(re.findall('<.*?>', sent))):
# #         print(i)
# #         print(sent)
#         break
#     i += 1

stop = set(stopwords.words('english')) #set of stopwords
print(len(stop))
stop.remove('not')
print(len(stop))


179
178


In [14]:
stop = set(stopwords.words('english')) #set of stopwords
stop.remove('not')
sno = nltk.stem.SnowballStemmer('english') #initialising the snowball stemmer

def cleanhtml(sentence): #function to clean the word of any html-tags
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, ' ', sentence)
    return cleantext
def cleanpunc(sentence): #function to clean the word of any punctuation or special characters
    cleaned = re.sub(r'[?|!|\'|"|#]',r'',sentence)
    cleaned = re.sub(r'[.|,|)|(|\|/]',r' ',cleaned)
    return  cleaned

In [15]:
#Code for implementing step-by-step the checks mentioned in the pre-processing phase
# this code takes a while to run as it needs to run on 500k sentences.
i=0
str1=' '
final_string=[]
all_positive_words=[] # store words from +ve reviews here
all_negative_words=[] # store words from -ve reviews here.
s=''
for sent in final['Text'].values:
    filtered_sentence=[]
    #print(sent);
    sent=cleanhtml(sent) # remove HTMl tags
    for w in sent.split():
        for cleaned_words in cleanpunc(w).split():
            if((cleaned_words.isalpha()) & (len(cleaned_words)>2)):    
                if(cleaned_words.lower() not in stop):
                    s=(sno.stem(cleaned_words.lower())).encode('utf8')
                    filtered_sentence.append(s)
                    if (final['Score'].values)[i] == 1: #'positive': 
                        all_positive_words.append(s)    #list of all words used to describe positive reviews
                    if(final['Score'].values)[i] == 0:  #'negative':
                        all_negative_words.append(s) #list of all words used to describe negative reviews reviews
                else:
                    continue
            else:
                continue 
    #print(filtered_sentence)
    str1 = b" ".join(filtered_sentence) #final string of cleaned words
    #print("***********************************************************************")
    
    final_string.append(str1)
    i+=1

In [16]:
final['CleanedText']=final_string #adding a column of CleanedText which displays the data after pre-processing of the review 

In [17]:

# store final table into an SQlLite table for future.
# conn = sqlite3.connect('final.sqlite')
# c=conn.cursor()
# conn.text_factory = str
# final.to_sql('Reviews', conn, schema=None, if_exists='replace', index=True, index_label=None, chunksize=None, dtype=None)

## Converting AFR data into IMDB format

In [18]:
final['Score'].value_counts()

1    307061
0     57110
Name: Score, dtype: int64

### As dataset is imbalanced, sampled equal datapoints for both the classes

In [19]:
# Sampling equal datapoints
final = shuffle(final)
final_balanced_reviews = final.loc[final['Score'] == 1][:57000]
neg_reviews = final.loc[final['Score'] == 0][:57000]
final_balanced_reviews = final_balanced_reviews.append(neg_reviews, ignore_index=True)
final_balanced_reviews = final_balanced_reviews[['Text', 'Score']]
final_balanced_reviews = shuffle(final_balanced_reviews)
final_balanced_reviews.head()

,Text,Score
113446,Received my package today and I was not please...,0
73475,"Used it to color on white chocolate candy, col...",0
92132,"The initial link states 12 - 20oz bottles, but...",0
46159,Imagine if you will a 1st grader in 1965.<br /...,1
31976,"Just what we wanted, loose rooibos tea. Green...",1


In [20]:
final_balanced_reviews['Score'].value_counts()

1    57000
0    57000
Name: Score, dtype: int64

## TF-IDF vectorization

In [55]:
X = final_balanced_reviews['Text']
Y = np.array(final_balanced_reviews['Score'])

In [56]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state = 123)
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size = 0.2, random_state = 123)

In [57]:
print('train size ->', X_train.shape, Y_train.shape)
print('test size ->', X_test.shape, Y_test.shape)
print('validation size ->', X_val.shape, Y_val.shape)

train size -> (72960,) (72960,)
test size -> (22800,) (22800,)
validation size -> (18240,) (18240,)


## TF-IDF featurization after train test split to avoid Data leakage problem

In [ ]:
train_vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_val = vectorizer.fit_transform(X_val)
print(type(X))
print(vectorizer.get_feature_names())

### Logistic Regression Best Hyperparameter search using GridSearch

In [47]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression


tuned_parameters = [{'C' : [10**-4, 10**-2, 10**0, 10**2, 10**4]}]

# Using GridSearchCV
model = GridSearchCV(LogisticRegression(), tuned_parameters, scoring='f1', cv=5)
model.fit(X_train, Y_train)

print('Best estimators ->', model.best_estimator_)
X_test = vectorizer.fit_transform(X_test)
print('SCore ->', model.score(X_test, Y_test))


C:\Users\Shubham\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Best estimators -> LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)
SCore -> 0.8938323160938001


### Logistic Regression on AFR 

In [48]:
from sklearn.linear_model import LogisticRegression
from sklearn.externals import joblib

clf = LogisticRegression(random_state = 0, solver = 'liblinear').fit(X_train, Y_train)
y_val_predict = clf.predict(X_val)
y_test_predict = clf.predict(X_test)

print("----Scores------")
print("train score ->", clf.score(X_train, Y_train))
print("test score ->", clf.score(X_test, Y_test))
print("validation score ->", clf.score(X_val, Y_val))


filename = 'LR_model.sav'
joblib.dump(clf, filename)

----Scores------
train score -> 0.9152428646242472
test score -> 0.8945773524720894
validation score -> 0.8917910447761194


['LR_model.sav']

### Naive Bayes on AFR

In [41]:
from sklearn.naive_bayes import GaussianNB, MultinomialNB

clf = MultinomialNB()
clf.fit(X_train, Y_train)
y_val_predict = clf.predict(X_val)
y_test_predict = clf.predict(X_test)

print("----Scores------")
print("train score ->", clf.score(X_train, Y_train))
print("test score ->", clf.score(X_test, Y_test))
print("validation score ->", clf.score(X_val, Y_val))

----Scores------
train score -> 0.8996301387797853
test score -> 0.8717171717171717
validation score -> 0.8695339094003666


### SVM on AFR

In [0]:
from sklearn.svm import SVC

clf = SVC(gamma = 'auto')
clf.fit(X_train, Y_train)
y_val_predict = clf.predict(X_val)
y_test_predict = clf.predict(X_test)

print("----Scores------")
print("train score ->", clf.score(X_train, Y_train))
print("test score ->", clf.score(X_test, Y_test))
print("validation score ->", clf.score(X_val, Y_val))

# Conclusions:

- Used total of 114,000 reviews from AFR dataset for this exercise.
- Balanced the dataset by taking 57k positive and 57K negative reviews.
- Converted AFR data into IMDB dataset format.
- Used index values of words to encode reviews into vectors.
- With single layer of accuarcy I am getting 100% accuracy.
- atanh activation performs better then relu in this case.
- Used return_sequences=True for 2 layers of LSTM.